In [1]:
import pandas as pd
import numpy as np
import os
import re
import json as js
from pathlib import Path
from tqdm import tqdm

In [11]:
# ganti directory untuk menjalankan program
directory_JSON_raw = "[Full] Bentuk JSON"
# directory_kamus = "Daftar Kamus Analisis Machine Readable"
directory_kamus = "[Full] Daftar Kamus Ekstraksi"
directory_hasil = "[Full] Raw CSV JSON all information"

list_json_raw = os.listdir(directory_JSON_raw)
list_directory_kamus = os.listdir(directory_kamus)

### Algoritma Ekstraksi Informasi JSON ###

Note : 1 page terdiri dari beberapa block, 1 block terdiri dari beberapa line, 1 line terdiri dari beberapa spans, dan 1 span terdiri dari beberapa detail

In [3]:
def extract_information_json(data):
    result = {
        "text":[],
        "size":[],
        "font":[],
        "x0":[], #[0]
        "y0":[], #[1]
        "x1":[], #[2]
        "y1":[], #[3]
        "page":[]
    }
    
    lines = []
    line_pages = []
    for key in data:
        page = data.get(key)
        for block in page.get("text").get("blocks"):
            if block.get("lines") is not None:
                lines.append(block.get("lines"))
                line_pages.append(page.get("page")) 
    
    spans = []
    span_pages = []
    for i in range(len(lines)):
        ln = lines[i]
        for detail_ln in ln:
            if detail_ln.get("spans") is not None:
                spans.append(detail_ln.get("spans"))
                span_pages.append(line_pages[i])
    
    for i in range(len(spans)):
        span = spans[i]
        for detail in span:
            result.get("text").append(detail.get("text"))
            result.get("size").append(detail.get("size"))
            result.get("font").append(detail.get("font"))
            result.get("x0").append(round(detail.get("bbox")[0],2))
            result.get("y0").append(round(detail.get("bbox")[1],2))
            result.get("x1").append(round(detail.get("bbox")[2],2))
            result.get("y1").append(round(detail.get("bbox")[3],2))
            result.get("page").append(span_pages[i]+1)
    
    return result

In [4]:
def is_contain_only_whitespaces(s):
    if re.match(r'^\s*$', str(s)): return True
    return False

def clean_information(data):
    result = {
        "text":[],
        "size":[],
        "font":[],
        "x0":[], #[0]
        "y0":[], #[1]
        "x1":[], #[2]
        "y1":[], #[3]
        "page":[]
    }
    
    i = 0
    while i < len(data["text"]):
        text = data["text"][i]
        if is_contain_only_whitespaces(text): # skip kata yang hanya mengandung whitespace
            i += 1
        else:
            result["text"].append(data["text"][i].strip())
            result["size"].append(data["size"][i])
            result["font"].append(data["font"][i])
            result["x0"].append(data["x0"][i])
            result["y0"].append(data["y0"][i])
            result["x1"].append(data["x1"][i])
            result["y1"].append(data["y1"][i])
            result["page"].append(data["page"][i])
            i += 1
            
    return result

In [5]:
"adap , halooo".split(",",1)

['adap ', ' halooo']

In [6]:
["adap",",","halo"][:2]

['adap', ',']

In [7]:
def seperate_span(data): # tokenisasi span berdasarkan spasi
    result = {
        "text":[],
        "size":[],
        "font":[],
        "x0":[], #[0]
        "y0":[], #[1]
        "x1":[], #[2]
        "y1":[], #[3]
        "page":[]
    }
    
    i = 0
    while i < len(data["text"]):
        curr_font = data["font"][i].lower()
        list_txt = data["text"][i].strip().split(" ")
        
        selisih_x = data["x1"][i] - data["x0"][i] # x1 - x0
        satuan_x = selisih_x/len(data["text"][i])
                
        if len(list_txt)==1: # jika span ternyata hanya satu kata
            result["text"].append(data["text"][i].strip())
            result["size"].append(data["size"][i])
            result["font"].append(data["font"][i])
            result["x0"].append(data["x0"][i])
            result["y0"].append(data["y0"][i])
            result["x1"].append(data["x1"][i])
            result["y1"].append(data["y1"][i])
            result["page"].append(data["page"][i])
            i+=1
        else:
            x0 = data["x0"][i]
            x1 = data["x1"][i] - len(data["text"][i])*satuan_x
            x1 = round(x1,2)

            y0 = data["y0"][i] # koordinat y tetap
            y1 = data["y1"][i]

            for txt in list_txt:
                x1 += len(txt) * satuan_x
                x1 = round(x1,2)

                result["text"].append(txt.strip())
                result["size"].append(data["size"][i])
                result["font"].append(data["font"][i])
                result["x0"].append(x0)
                result["y0"].append(y0)
                result["x1"].append(x1)
                result["y1"].append(y1)
                result["page"].append(data["page"][i])

                x0 += (len(txt)+1) * satuan_x
                x0 = round(x0,2)
            i+=1
            
        
    return result

In [8]:
def is_contain_bold_and_italic(font):
    contains_bold = False; contains_italic = False
    for i in font:
        if "bold" in i.lower(): contains_bold = True
        if "italic" in i.lower(): contains_italic = True
        if contains_bold == True and contains_italic == True: return True
    return False

def is_contain_bold(font):
    contains_bold = False
    for i in font:
        if "bold" in i.lower(): contains_bold = True
    return contains_bold

def count_bold(font):
    cnt = 0
    for i in font:
        if "bold" in i.lower(): 
            cnt += 1
    return cnt

In [9]:
daftar_kamus_layak = []

daftar_kamus = os.listdir(directory_JSON_raw)
daftar_kamus = sorted(daftar_kamus)
for filename in tqdm(daftar_kamus):
    print("====" + filename + "====")
    with open(directory_JSON_raw + "/" + filename,"rb") as f:
        data = js.load(f)
    f.close()
    
    new_filename = os.path.splitext(filename)[0]
    result_raw = extract_information_json(data)
    result_clean = clean_information(result_raw)
    result = seperate_span(result_clean)
    
    if (is_contain_bold_and_italic(result["font"])):
        csv_res = pd.DataFrame.from_dict(result)
        csv_res.to_csv(directory_hasil + "/" + new_filename + "-ekstraksi.csv",index=False)
        daftar_kamus_layak.append(new_filename)

 12%|█████████▊                                                                         | 2/17 [00:00<00:00, 17.30it/s]

====15. Kamus Bahasa Indonesia-Pasir (1997)-page_14_15-hasil.json====
====22. Kamus Bahasa Gorontalo-Indonesia (1977)-page_108_109-hasil.json====
====39. Kamus Alas Indonesia (1985)-page_61_62-hasil.json====
====41. Kamus Bahasa Indonesia-Bali A-K (1997)-page_58_59-hasil.json====


 35%|█████████████████████████████▎                                                     | 6/17 [00:00<00:00, 13.51it/s]

====43. Kamus Bahasa Indonesia-Bahasa Minangkabau I (1994)-page_23_24-hasil.json====
====53. Kamus Bakumpai Indonesia (1985)-page_38_39-hasil.json====
====54. Kamus Bahasa Indonesia Mentawai (1998)-page_23_24-hasil.json====
====57. Kamus Bahasa Bugis-Indonesia (1977)-page_20_21-hasil.json====


 71%|█████████████████████████████████████████████████████████▉                        | 12/17 [00:00<00:00, 20.12it/s]

====61. Kamus Banjar-Indonesia (1977)-page_51_52-hasil.json====
====63. Kamus Bahasa Indonesia-Lampung Dialek A (1999)-page_97_98-hasil.json====
====68. Kamus Dwibahasa Bahasa Talaud - Bahasa Indonesia (2018)-page_19_20-hasil.json====
====77. Kamus Samawa-Indonesia Edisi 2 (2017)-page_27_28-hasil.json====
====82. Kamus dwibahasa Indonesia - Aceh (2011)-page_25_26-hasil.json====
====83. Kamus dwibahasa Bahasa Indonesia - Dayak Halong (2017)-page_14_15-hasil.json====


 88%|████████████████████████████████████████████████████████████████████████▎         | 15/17 [00:00<00:00, 18.51it/s]

====86. Kamus Bahasa Indonesia-Minangkabau edrevisi (2013)-page_34_35-hasil.json====
====89. Kamus Dwibahasa Bahasa Mooi-Bahasa Indonesia (2017)-page_17_18-hasil.json====
====91. Kamus Simalungun - Indonesia (edisi kedua) (2015)-page_48_49-hasil.json====


100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:01<00:00, 16.93it/s]


In [10]:
print("==== Daftar Kamus Layak ====")

for i in daftar_kamus_layak:
    print(i)

==== Daftar Kamus Layak ====
1. Kamus Makasar-Indonesia (1995)-hasil
10. Kamus Bahasa Indonesia-Dayak Deah Edisi I (2013)-hasil
11. Kamus Suwawa-Indonesia (1985)-hasil
12. Kamus Bahasa Indonesia-Kaidipang L-Z (2000)-hasil
13. Kamus Bahasa Indonesia-Bahasa Sunda I (1993)-hasil
14. Kamus Bahasa Indonesia-Bahasa Minangkabau II (1994)-hasil
15. Kamus Bahasa Indonesia-Pasir (1997)-hasil
16. Kamus Bahasa Indonesia Karo A-K (1998)-hasil
17. Kamus Melayu Makasar-Indonesia (1985)-hasil
18. Kamus Bahasa Jawa-Bahasa Indonesia I (1993)-hasil
19. Kamus Bahasa Indoensia-Melayu Riau (1997)-hasil
2. Kamus Melayu-Indonesia (1985)-hasil
20. Kamus Bahasa Melayu Ambon-Indonesia (1998)-hasil
21. Kamus Bahasa Indonesia-Sentani A-K (1999)-hasil
22. Kamus Bahasa Gorontalo-Indonesia (1977)-hasil
23. Kamus Dwibahasa Dayak Ngaju-Indonesia (2013)-hasil
24. Kamus Minangkabau-Indonesia (1985)-hasil
25. Kamus Bahasa Indonesia Jambi L-Z (1997)-hasil
26. Kamus Bahasa Indonesia-Bahasa Tonsea II (1996)-hasil
27. Kamus B

# Pembersihan untuk kamus full

* kamus 10 -> >= 16 dan <= 315
* kamus 11 -> >= 26 dan <= 360
* kamus 12 -> >= 18 dan <= 529
* kamus 13 -> >= 24 dan <= 431
* kamus 14 -> >= 6 dan <= 470
* kamus 15 -> >= 18 dan <= 546
* kamus 16 -> >= 18 dan <= 418
* kamus 17 -> >= 12 dan <= 154
* kamus 18 -> >= 20 dan <= 469
* kamus 19 -> >= 27 dan <= 462
* kamus 2 -> >= 8 dan <= 237
* kamus 20 -> >= 16 dan <= 153
* kamus 21 -> >= 16 dan <= 197
* kamus 22 -> >= 38 dan <= 339
* kamus 23 -> >= 26 dan <= 179
* kamus 24 -> 22, 333
* kamus 25 -> 8, 339
* kamus 26 -> 10, 177
* kamus 27 -> 24, 143
* kamus 28 -> 36, 471
* kamus 29 -> 40, 157
* kamus 3 -> 38, 339
* kamus 31 -> 34, 200
* kamus 32 -> 12, 194
* kamus 33 -> 15, 205
* kamus 34 -> 20, 517
* kamus 36 -> 23, 244 
* kamus 37 -> 8, 257
* kamus 38 -> 18, 601
* kamus 4 -> 18, 225
* kamus 41 -> 18, 468
* kamus 42 -> 10, 495
* kamus 44 -> 8, 124
* kamus 46 -> 18, 327
* kamus 48 -> 16, 155
* kamus 5 -> 10, 158
* kamus 50 -> 10, 181
* kamus 51 -> 26, 147
* kamus 52 -> 14, 262
* kamus 54 -> 16, ~
* kamus 55 -> 12, 209
* kamus 56 -> 18, 322
* kamus 58 -> 10, 168
* kamus 60 -> 68, 449 
* kamus 63 -> 12, 332
* kamus 66 -> 46, 171
* kamus 67 -> 24, 583
* kamus 68 -> 35, 317
* kamus 70 -> 14, 93
* kamus 71 -> 15, 63
* kamus 74 -> 23, 178 
* kamus 78 -> 12, 171
* kamus 79 -> 6, 420
* kamus 8 -> 10, 163
* kamus 82 -> 21, 203
* kamus 83 -> 13, 251 // tidak termasuk abjad y
* kamus 84 -> 10, 88
* kamus 85 -> 37, 335
* kamus 87 -> 19, 543
* kamus 89 -> 26, 103
* kamus 9 -> 21, 156
* kamus 91 -> 19, 282
* kamus 94 -> 38, 245
----------------------------------

In [12]:
split_kamus = pd.read_csv("Split Kamus.csv")

kamus_split = split_kamus["Kamus"].values.tolist()

awal = split_kamus["halaman_pertama"].values.tolist()

akhir = split_kamus["halaman_terakhir"].values.tolist()

In [14]:
daftar_kamus_tidak_layak = []
daftar_kamus_layak_setelah_displit = []

directory_hasil_raw = os.listdir(directory_hasil)

for i in range(len(directory_hasil_raw)):
    filename_clean = directory_hasil_raw[i]
    print("====" + filename_clean + "====" + str(kamus_split[i]))
    
    kamus = pd.read_csv(directory_hasil + "/" + filename_clean)
    kamus = kamus[kamus["page"] >= awal[i]]
    kamus = kamus[kamus["page"] <= akhir[i]]
    kamus = kamus.reset_index(drop=True)
    
    if is_contain_bold(kamus["font"].values.tolist()):
        print("==== Memenuhi Kriteria ====")
        directory_hasil_clean = "[Full] CSV JSON all information"
        kamus.to_csv(directory_hasil_clean + "/" + filename_clean,index=False)
        daftar_kamus_layak_setelah_displit.append(filename_clean)
    else:
        print("==== Tidak Memenuhi Kriteria ====")
        daftar_kamus_tidak_layak.append(filename_clean)
    

====10. Kamus Bahasa Indonesia-Dayak Deah Edisi I (2013)-hasil-ekstraksi.csv====10
==== Memenuhi Kriteria ====
====11. Kamus Suwawa-Indonesia (1985)-hasil-ekstraksi.csv====11
==== Memenuhi Kriteria ====
====12. Kamus Bahasa Indonesia-Kaidipang L-Z (2000)-hasil-ekstraksi.csv====12
==== Memenuhi Kriteria ====
====13. Kamus Bahasa Indonesia-Bahasa Sunda I (1993)-hasil-ekstraksi.csv====13
==== Memenuhi Kriteria ====
====14. Kamus Bahasa Indonesia-Bahasa Minangkabau II (1994)-hasil-ekstraksi.csv====14
==== Memenuhi Kriteria ====
====15. Kamus Bahasa Indonesia-Pasir (1997)-hasil-ekstraksi.csv====15
==== Memenuhi Kriteria ====
====16. Kamus Bahasa Indonesia Karo A-K (1998)-hasil-ekstraksi.csv====16
==== Memenuhi Kriteria ====
====17. Kamus Melayu Makasar-Indonesia (1985)-hasil-ekstraksi.csv====17
==== Memenuhi Kriteria ====
====18. Kamus Bahasa Jawa-Bahasa Indonesia I (1993)-hasil-ekstraksi.csv====18
==== Memenuhi Kriteria ====
====19. Kamus Bahasa Indoensia-Melayu Riau (1997)-hasil-ekstraksi

In [15]:
for i in daftar_kamus_tidak_layak:
    print(i)

82. Kamus dwibahasa Indonesia - Aceh (2011)-hasil-ekstraksi.csv
83. Kamus dwibahasa Bahasa Indonesia - Dayak Halong (2017)-hasil-ekstraksi.csv


In [16]:
for i in daftar_kamus_layak_setelah_displit:
    print(i)

10. Kamus Bahasa Indonesia-Dayak Deah Edisi I (2013)-hasil-ekstraksi.csv
11. Kamus Suwawa-Indonesia (1985)-hasil-ekstraksi.csv
12. Kamus Bahasa Indonesia-Kaidipang L-Z (2000)-hasil-ekstraksi.csv
13. Kamus Bahasa Indonesia-Bahasa Sunda I (1993)-hasil-ekstraksi.csv
14. Kamus Bahasa Indonesia-Bahasa Minangkabau II (1994)-hasil-ekstraksi.csv
15. Kamus Bahasa Indonesia-Pasir (1997)-hasil-ekstraksi.csv
16. Kamus Bahasa Indonesia Karo A-K (1998)-hasil-ekstraksi.csv
17. Kamus Melayu Makasar-Indonesia (1985)-hasil-ekstraksi.csv
18. Kamus Bahasa Jawa-Bahasa Indonesia I (1993)-hasil-ekstraksi.csv
19. Kamus Bahasa Indoensia-Melayu Riau (1997)-hasil-ekstraksi.csv
2. Kamus Melayu-Indonesia (1985)-hasil-ekstraksi.csv
20. Kamus Bahasa Melayu Ambon-Indonesia (1998)-hasil-ekstraksi.csv
21. Kamus Bahasa Indonesia-Sentani A-K (1999)-hasil-ekstraksi.csv
22. Kamus Bahasa Gorontalo-Indonesia (1977)-hasil-ekstraksi.csv
23. Kamus Dwibahasa Dayak Ngaju-Indonesia (2013)-hasil-ekstraksi.csv
24. Kamus Minangkabau-